In [ ]:
import pandas as pd
import numpy as np
import pickle


In [ ]:
X_train = pd.read_csv('train_.csv', index_col='SK_ID_CURR')
X_val = pd.read_csv('val_.csv', index_col='SK_ID_CURR')

In [ ]:
with open('catb.pkl', 'rb') as f:
    catb = pickle.load(f)

## Пороги, условия на признаки и суммы

Посчитаем квартили вероятностей, которые предсказала модель по обучающей выборке, отдельно для обоих классов.

In [ ]:
zeros = np.where(X_train['TARGET'] == 0)
q0 = pd.DataFrame(catb.predict_proba(X_train)[:, 1][zeros], columns=['target = 0']).quantile([0.25, 0.5, 0.75, 0.9])

ones = np.where(X_train['TARGET'] == 1)
q1 = pd.DataFrame(catb.predict_proba(X_train)[:, 1][ones], columns=['target = 1']).quantile([0.25, 0.5, 0.75])

pd.concat([q0, q1], axis=1)

,target = 0,target = 1
0.25,0.028935,0.082745
0.50,0.051005,0.153652
0.75,0.092250,0.277561
0.90,0.151437,NaN


Возьмем порог в 0.15.

Таким образом мы пропускаем меньше половины дефолтников (а их в общее количество в выборке намного меньше добросовестных клиентов), и в тоже время пропускаем большую часть добросовестных заемщиков.

Промежуточные пороги возьмем такие: 0.02, 0.05, 0.09, 0.12.

Затем я стал смотреть, как распределены некоторые признаки (рассматривал не все, так как их вышло четыре сотни), когда вероятности принимают значения в этих промежутках, пытаясь найти способ отсеять дефолтных клиентов (все еще используя обучающую выборку). 

Как можно было ожидать, почти всегда признаки у обоих классов принимали примерно одни и те же значения, хоть и нашлась парочка отклонений. 

Условие на ext_score_weighed взялось из следующего наблюдения, но теперь по всей обучающей выборке:

In [ ]:
pd.concat([
    X_train[X_train['TARGET'] == 0]['ext_score_weighed'].quantile([0.25, 0.5, 0.75]),
    X_train[X_train['TARGET'] == 1]['ext_score_weighed'].quantile([0.25, 0.5, 0.75])
], axis=1)


,ext_score_weighed,ext_score_weighed
0.25,0.527712,0.354113
0.50,0.718965,0.525513
0.75,0.880774,0.702905


Также смотрел на то сколько в данном интервале клиентов каждого класса. 

Это же еще использовалось для выбора сумм. Их я взял поменьше, так как смотреть на все эти нули при выводе доходов и убытков было невозможно.

Все это дело выбиралось и отбрасывалось (преимущественно методом тыка), так чтобы максимизировать отношение фаткического дохода к потенциальному.

Учитывалось также то, чтобы сам доход не принимал слишком низких значений. 

Процентная ставка:

In [ ]:
X_train['rate'].mean()

12.365762494815298

In [ ]:
# возьмем объекты из ядра
import sys
sys.path.append('./git/')

from src.app.core.api import (
    Features
)
from src.app.core.model import Model

Посчитаем доходы и убытки на трейне и на валидации.

Для этого заведем следующую функцию.

In [ ]:
def get_info_results(df, model):
    """Присоединяет к датасету результаты скоринга и выводит статистику по выдаче."""
    df['result'] = df.apply(
        lambda x: model.get_scoring_result(
            Features(
                x.drop('TARGET'),
                
                x['avg_income_per_child'],
                x['ext_score_weighed'],
                x['ratio_open']
            )
        ),
        axis=1
    )
    df['decision'] = df['result'].apply(lambda x: x.decision.name)
    df['amount'] = df['result'].apply(lambda x: x.amount)
    df['proba'] = df['result'].apply(lambda x: x.proba)
    
    s1 = df[(df['decision'] == 'accepted')]['amount'].sum()
    print('выдано:', s1)
    
    s2 = df[(df['decision'] == 'accepted') & (df['TARGET'] == 0)]['amount'].sum() * 0.12 
    print('прибыль:', s2)
    
    s3 = df[(df['decision'] == 'accepted') & (df['TARGET'] == 1)]['amount'].sum()
    print('убытки (то есть сколько не вернули):', s3)
    
    print('доход:', s2-s3)
    
    s4 = df[(df['decision'] == 'accepted')]['amount'].sum() * 0.12
    print('потенциальный доход:', s4)
    
    print('отношение фактического дохода к потенциальному:', (s2-s3)/s4)
    
    return df

In [ ]:
model = Model('catb.pkl')

In [ ]:
X_train = get_info_results(X_train, model)

# сразу добавим колонку с вероятностью "голой" модели
# CatBoost прежде, чем смотреть использовалась ли эта колонка при обучении, переводит ее в числа. 
# И вот с этими двумя колонками у него проблемы, поэтому их дропаем.
X_train['proba_original'] = catb.predict_proba(X_train.drop(['result', 'decision'], axis=1))[:, 1]

выдано: 11189930
прибыль: 1302664.2
убытки (то есть сколько не вернули): 334395
доход: 968269.2
потенциальный доход: 1342791.5999999999
отношение фактического дохода к потенциальному: 0.7210867270840837


In [ ]:
X_val = get_info_results(X_val, model)

# сразу добавим колонку с вероятностью "голой" модели
# CatBoost прежде, чем смотреть использовалась ли эта колонка при обучении, переводит ее в числа. 
# И вот с этими двумя колонками у него проблемы, поэтому их дропаем
X_val['proba_oroginal'] = catb.predict_proba(X_val.drop(['result', 'decision'], axis=1))[:, 1]

выдано: 4815980
прибыль: 557475.0
убытки (то есть сколько не вернули): 170355
доход: 387120.0
потенциальный доход: 577917.6
отношение фактического дохода к потенциальному: 0.6698532801215952


Результат на валидационной выборке, конечно, не такой же как на трейне, но вроде бы близок.